# Testing the normal label formulation and the DL Enhancement / Lifted Formulation from https://link.springer.com/content/pdf/10.1007/s13676-012-0010-0.pdf

In [1]:
import sys, os
sys.path.append(os.path.abspath('..'))
from imports import ilpgraph, readFile
import tsplib95
from imports import networkx as imp_nx
from network import min_tsp, gen_path_atsp, atsp_DT_formulation, tsp_callbacks
import networkx as nx
from gurobipy import *
import xlsxwriter
import random
import matplotlib.pyplot as plt



def __write_solution(solution:list):
    print("writing solutions...")
    workbook = xlsxwriter.Workbook('BenchmarksTSP.xlsx')
    worksheet = workbook.add_worksheet()
    bold = workbook.add_format({'bold':True})
    solutionRows = len(solution)
    solutionColumns = len(solution[0])
    
    worksheet.write('A1', 'Instance', bold)
    worksheet.write('B1', 'Edges', bold)
    worksheet.write('C1', 'Nodes', bold)
    worksheet.write('E1', 'Runtime', bold)
    worksheet.write('F1', 'Objective Value', bold)
    worksheet.write('G1', 'Constraints', bold)
    worksheet.write('H1', 'MIP Gap', bold)
    
    for i in range(solutionRows):
        for j in range(solutionColumns):
            worksheet.write(i + 1, j, solution[i][j])
    
        
def runBenchmark(paths:list):
    solutions = []
    i = 0
    
    for path in paths:
        tsp = tsplib95.load(path)
        
        G = tsp.get_graph()
        G = imp_nx.read(G)
        #m = gen_path_atsp.createGenModel(G,'min','metric')
        #m = atsp_DT_formulation.createGenModel(G, "min", "metric")
        m = tsp_callbacks.createGenModel(G, "min", "metric")
        m.params.MIPGap = 0.05
        m.params.TimeLimit = 1500
       

        m.optimize()

        solutions.append((i, len(G.G.edges()), len(G.G.nodes()), m.runTime, m.objVal, m.numConstrs, m.MIPGap))
        i += 1
    print(solutions)
    __write_solution(solutions)         

if __name__ == '__main__':
    

    paths = ["TSPTestInstances/p654.tsp"]
    
    runBenchmark(paths)


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2021-03-23
Using license file C:\Program Files\Gurobi\gurobi.lic
Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 4965 rows, 2739025 columns and 5479705 nonzeros
Model fingerprint: 0x1a850a53
Variable types: 0 continuous, 2739025 integer (2739025 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
In Callback
Presolve removed 1655 rows and 1655 columns (presolve time = 6s) ...
Presolve removed 1655 rows and 1655 columns
Presolve time: 7.80s
Presolved: 3310 rows, 2737370 columns, 5474740 nonzeros
Variable types: 0 continuous, 2737370 integ

GurobiError: Out of memory

# Heuristic implementation. Using Nearest Neighbour to create a new Tour and 2 Opt to improve the tour

In [34]:
import sys, os
sys.path.append('../..')
sys.path.append('../../..')
from imports import ilpgraph, readFile
from imports import networkx as imp_nx
from network.heuristics import NearestNeighbour as NN
from network.heuristics import twoOpt
import networkx as nx
from gurobipy import *
import xlsxwriter
import random
import matplotlib.pyplot as plt
import tsplib95

def noRound(x):
    return x

# Specification on amount of cities
nbr_cities = 150
# Path of the TSPLib Instance
path = r"TSPTestInstances/kroB150.tsp"
# Loading problem via tsplib95. Returns a dictionary with a bunch of information that the code doesn't need.
problem = tsplib95.load(path).as_dict()
distances = [[]]

# Constructing distance matrix
for i in range(nbr_cities):
    for j in range(nbr_cities):
        # If the nodes are the same, i.e. an entry on the diagonal, add extremely high costs.
        # This way, routes from a route to itself will never be profitable
        if (i == j):
            distances[i].append(10000)
        # Else calculate the euclidean distance by tsplib function
        else:
            distances[i].append(tsplib95.distances.euclidean(problem['node_coords'][i+1], problem['node_coords'][j+1], noRound))
    if (i == nbr_cities-1):
        break
    distances.append([])
    
# Calculate Nearest Neighbours
tour, length = NN.nearNeighHeur(distances, nbr_cities)
print("Nearest Neighbor Tour ", tour)
print("Tour length ", length)
print("2 OPT")
# Improve with 2 OPT Heuristic
tour, length = twoOpt.getSol(distances, tour, length, nbr_cities)
print(tour)
print(length)



Nearest Neighbor Tour  [0, 52, 84, 26, 14, 12, 78, 109, 63, 19, 54, 41, 66, 30, 46, 5, 148, 53, 133, 74, 21, 111, 7, 105, 16, 24, 89, 33, 144, 108, 97, 147, 87, 27, 38, 37, 70, 123, 71, 82, 61, 137, 112, 107, 113, 125, 100, 55, 118, 4, 42, 85, 132, 49, 94, 93, 122, 90, 69, 110, 120, 75, 130, 138, 20, 121, 88, 40, 58, 2, 72, 117, 68, 73, 56, 35, 99, 32, 44, 80, 96, 143, 135, 9, 91, 98, 140, 18, 104, 142, 92, 48, 17, 28, 36, 119, 15, 134, 145, 126, 50, 62, 43, 65, 102, 127, 83, 10, 51, 149, 95, 86, 47, 13, 3, 115, 59, 139, 114, 76, 79, 146, 64, 129, 106, 103, 1, 34, 67, 29, 22, 101, 128, 57, 39, 11, 45, 124, 141, 131, 31, 116, 23, 60, 25, 77, 8, 81, 6, 136, 0]
Tour length  32825.749887629725
2 OPT
32780.97760798701
32582.406815106897
32521.826023462298
32466.899095821867
32439.635218454434
32344.806780933493
32101.87846689871
32101.043943278117
32098.65167897882
31930.750646371645
31831.657416181686
31638.671961263168
31571.833600768234
31552.555947148332
31342.939182306745
31148.7146483

KeyboardInterrupt: 